In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
#plt.rcParams['font.family'] = 'AppleGothic' # 폰트 변경
#plt.rcParams['axes.unicode_minus'] = False # 축 값 마이너스 깨짐 해결

from torch.nn import Transformer
from torch import nn
import torch
import math
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_percentage_error ,mean_squared_error

In [110]:
#path = '~/workplace/datas/energy/'
path = '~/Desktop/workplace/datas/energy/'

train_df = pd.read_csv(path + '기상변수_전력수요량_5min.csv',index_col=0)
y_test = pd.read_csv(path + 'y_test.csv', index_col=0)

In [111]:
train_df.reset_index(inplace=True)

In [112]:
train_df = train_df.rename(columns={'index':'기준일시'})

In [113]:
train_df.tail()

,기준일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),적설(cm),전운량(10분위),지면온도(°C),현재수요(MW)
547516,2023-03-19 23:35:00,4.946842,0.0,0.862368,68.204386,1006.203070,0.08617,0.060784,0.0,2.612281,3.775088,58627.16
547517,2023-03-19 23:40:00,4.870526,0.0,0.864211,68.477193,1006.187719,0.08617,0.060784,0.0,2.645614,3.722807,57941.24
547518,2023-03-19 23:45:00,4.794211,0.0,0.866053,68.750000,1006.172368,0.08617,0.060784,0.0,2.678947,3.670526,57726.21
547519,2023-03-19 23:50:00,4.717895,0.0,0.867895,69.022807,1006.157018,0.08617,0.060784,0.0,2.712281,3.618246,57755.40
547520,2023-03-19 23:55:00,4.641579,0.0,0.869737,69.295614,1006.141667,0.08617,0.060784,0.0,2.745614,3.565965,57323.79


In [114]:
train_df['기준일시'] = pd.to_datetime(train_df['기준일시'])

In [115]:
train_df['hour'] = train_df['기준일시'].dt.hour
train_df['month'] = train_df['기준일시'].dt.month
train_df['year'] = train_df['기준일시'].dt.year
train_df['day'] = train_df['기준일시'].dt.day
train_df['min'] = train_df['기준일시'].dt.minute

In [ ]:
#더미를 같이하고 나눠줘야 컬럼수가 같구나....

In [117]:
train_df = pd.get_dummies(train_df, columns=['hour', 'month', 'year', 'day', 'min'])

In [118]:
train_df.shape

(547521, 97)

In [119]:
encoded_test_df = train_df[train_df['기준일시'] >= '2023-03-13']
encoded_train_df = train_df[train_df['기준일시'] < '2023-03-13']

In [120]:
encoded_test_df.drop(columns=['기준일시'], inplace=True)
encoded_train_df.drop(columns=['기준일시'], inplace=True)

/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_17942/542440738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_test_df.drop(columns=['기준일시'], inplace=True)
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_17942/542440738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_train_df.drop(columns=['기준일시'], inplace=True)


In [121]:
encoded_test_df.shape , encoded_train_df.shape

((2016, 96), (545505, 96))

In [122]:
encoded_train_df.to_csv(path + 'encoded_train_df.csv')
encoded_test_df.to_csv(path + 'encoded_test_df.csv')

# 7.18 data 

In [4]:
#path = '~/workplace/datas/energy/'
path = '~/Desktop/workplace/datas/energy/'

df = pd.read_csv(path + '기상변수_전력수요량_5min.csv',index_col=0)
df

,Year,Month,Day,Hour,기온(°C),풍속(m/s),습도(%),현지기압(hPa),지면온도(°C),강수량(mm),적설(cm),일조(hr),일사(MJ/m2),현재수요(MW)
2018-01-01 00:00:00,2018,1,1,0,-1.298947,1.595789,55.042105,1011.750526,-1.706316,0.0,0.034737,0.0,0.0,60955.00
2018-01-01 00:05:00,2018,1,1,0,-1.327281,1.603772,55.147368,1011.747632,-1.722982,0.0,0.034737,0.0,0.0,60964.30
2018-01-01 00:10:00,2018,1,1,0,-1.355614,1.611754,55.252632,1011.744737,-1.739649,0.0,0.034737,0.0,0.0,60745.90
2018-01-01 00:15:00,2018,1,1,0,-1.383947,1.619737,55.357895,1011.741842,-1.756316,0.0,0.034737,0.0,0.0,60170.50
2018-01-01 00:20:00,2018,1,1,0,-1.412281,1.627719,55.463158,1011.738947,-1.772982,0.0,0.034737,0.0,0.0,60067.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19 23:35:00,2023,3,19,23,4.946842,0.862368,68.204386,1006.203070,3.775088,0.0,0.000000,0.0,0.0,58627.16
2023-03-19 23:40:00,2023,3,19,23,4.870526,0.864211,68.477193,1006.187719,3.722807,0.0,0.000000,0.0,0.0,57941.24
2023-03-19 23:45:00,2023,3,19,23,4.794211,0.866053,68.750000,1006.172368,3.670526,0.0,0.000000,0.0,0.0,57726.21
2023-03-19 23:50:00,2023,3,19,23,4.717895,0.867895,69.022807,1006.157018,3.618246,0.0,0.000000,0.0,0.0,57755.40


In [5]:
#train_test split

encoded_test_df = df[df.index >= '2023-03-13']
encoded_train_df = df[df.index < '2023-03-13']

In [9]:
encoded_test_df.to_csv(path + 'encoded_test_df.csv')
encoded_train_df.to_csv(path + 'encoded_train_df.csv')

# 7월 19일 데이터

In [1]:
import pandas as pd

In [2]:
path = '~/workplace/datas/energy/'
#path = '~/Desktop/workplace/datas/energy/'

df = pd.read_csv(path + '기존데이터+하루전수요.csv',index_col=0)
df

,Year,Month,Day,Hour,기온(°C),풍속(m/s),습도(%),현지기압(hPa),지면온도(°C),강수량(mm),적설(cm),일조(hr),일사(MJ/m2),smp(원/kWh),발전량,휴일,5분전_현재수요(MW),1시간전_현재수요(MW),하루전_현재수요(MW),현재수요(MW)
일시,,,,,,,,,,,,,,,,,,,,
2018-01-02 01:05:00,2018,1,2,1,-2.955526,1.016140,67.514912,1015.174211,-3.007807,0.0,0.022105,0.0,0.0,91.991667,5246.400000,0,58703.40,60556.60,58194.60,58466.40
2018-01-02 01:10:00,2018,1,2,1,-2.994211,1.009123,67.735088,1015.189474,-3.031404,0.0,0.022105,0.0,0.0,91.723333,5246.400000,0,58466.40,60158.70,57830.60,58508.60
2018-01-02 01:15:00,2018,1,2,1,-3.032895,1.002105,67.955263,1015.204737,-3.055000,0.0,0.022105,0.0,0.0,91.455000,10428.000000,0,58508.60,59842.30,57745.30,58209.90
2018-01-02 01:20:00,2018,1,2,1,-3.071579,0.995088,68.175439,1015.220000,-3.078596,0.0,0.022105,0.0,0.0,91.186667,5731.200000,0,58209.90,59771.70,57329.60,58080.50
2018-01-02 01:25:00,2018,1,2,1,-3.110263,0.988070,68.395614,1015.235263,-3.102193,0.0,0.022105,0.0,0.0,90.918333,6254.400000,0,58080.50,59393.60,57219.70,57942.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19 23:35:00,2023,3,19,23,4.946842,0.862368,68.204386,1006.203070,3.775088,0.0,0.000000,0.0,0.0,211.199167,923693.055902,0,58949.76,59232.87,60557.22,58627.16
2023-03-19 23:40:00,2023,3,19,23,4.870526,0.864211,68.477193,1006.187719,3.722807,0.0,0.000000,0.0,0.0,219.743333,923693.055902,0,58627.16,59151.67,60279.80,57941.24
2023-03-19 23:45:00,2023,3,19,23,4.794211,0.866053,68.750000,1006.172368,3.670526,0.0,0.000000,0.0,0.0,228.287500,923693.055902,0,57941.24,59046.37,60260.97,57726.21


In [3]:
#train_test split

encoded_test_df = df[df.index >= '2023-03-13']
encoded_train_df = df[df.index < '2023-03-13']

In [4]:
encoded_test_df.shape , encoded_train_df.shape

((2016, 20), (546035, 20))

In [5]:
encoded_test_df.to_csv(path + 'encoded_test_df.csv')
encoded_train_df.to_csv(path + 'encoded_train_df.csv')